# Import Markdown files and notebooks

Literate `pidgy` programs are reusable as [Python] scripts and modules.
These features are configured by inheriting features from `importnb` that customize the [Python] import system to discover/load alternative source files. `pidgy` treats
[Python], [Markdown], and [Notebook] files as python source.

`sys.meta_path and sys.path_hooks`

In [1]:
    __all__ = 'pidgyLoader',
    import pidgy, IPython, importnb

`get_data` determines how a file is decoding from disk.  We use it to make an escape hatch for markdown files otherwise we are importing a notebook.

In [3]:
    def get_data(self, path):
        if self.path.endswith('.md'): return self.code(self.decode())
        return super(pidgyLoader, self).get_data(path)

The `code` method tangles the [Markdown] to [Python] before compiling to an [Abstract Syntax Tree].

In [4]:
    def code(self, str):
        for callable in (pidgyLoader.transformer_manager.transform_cell, 
                         pidgy.extras.demojize):
            str = ''.join(callable(''.join(str)))
        return str

The `visit` method allows custom [Abstract Syntax Tree] transformations to be applied.

In [6]:
    def visit(self, node):
        return pidgy.extras.ExtraSyntax().visit(node)

Attach these methods to the `pidgy` loader.

Only [Python] files and common flavored notebooks may be used as source code before
the `pidgyLoader` is defined. Once the `pidgyLoader` is defined [Markdown] becomes
a new source target for [Python] and [Notebook]s bearing the `".md.ipynb"` extension are consumed specially as `pidgy` flavored documents.

In [2]:
    class pidgyLoader(importnb.Notebook): 
        extensions = ".py.md .md .md.ipynb".split()
        transformer_manager = pidgy.tangle.pidgyManager()
        code = code
        visit = visit
        get_source = get_data = get_data